# Лекция 12: Кластеризация. Поиск структуры в данных без учителя

**Цель лекции:**
1. Понять разницу между обучением с учителем и **обучением без учителя**.
2. Изучить **K-Means** — самый популярный алгоритм кластеризации на основе центроидов.
3. Кратко познакомиться с **иерархической кластеризацией** и дендрограммами.
4. Изучить **DBSCAN** — алгоритм кластеризации на основе плотности.
5. Сравнить подходы, чтобы понимать, какой алгоритм для какой задачи подходит лучше.

### Часть 1: Введение в Обучение без учителя

До сих пор мы работали в парадигме **обучения с учителем (Supervised Learning)**. У нас всегда была размеченная обучающая выборка с признаками `X` и правильными ответами `y`. Наша цель была — построить модель, которая предсказывает `y` по `X`.

Теперь мы переходим к **обучению без учителя (Unsupervised Learning)**. Главное отличие — у нас есть **только признаки `X`**, а **правильных ответов `y` нет**. Задача — не предсказать метку, а найти скрытую структуру, закономерности или группы в самих данных.

#### 1.1. Типы задач обучения без учителя

Обучение без учителя — это целое семейство алгоритмов. Кластеризация — самый известный, но далеко не единственный его представитель.

1.  **Кластеризация (Clustering):**
    *   **Цель:** Сгруппировать похожие объекты вместе, а непохожие — разнести по разным группам (кластерам).
    *   **Пример:** Сегментация клиентов магазина по их покупательскому поведению.
    *   **Алгоритмы:** K-Means, DBSCAN, Иерархическая кластеризация.

2.  **Уменьшение размерности (Dimensionality Reduction):**
    *   **Цель:** Сократить количество признаков в датасете, сохранив при этом как можно больше полезной информации. Это помогает бороться с "проклятием размерности", ускорять обучение других моделей и визуализировать данные.
    *   **Пример:** Преобразовать датасет с 50 признаками в 2 признака, чтобы нарисовать его на 2D-графике.
    *   **Алгоритмы:** Метод главных компонент (PCA), t-SNE.

3.  **Поиск ассоциативных правил (Association Rule Learning):**
    *   **Цель:** Найти интересные взаимосвязи и закономерности между переменными в больших наборах данных.
    *   **Пример:** Анализ чеков в супермаркете для поиска правил вида "Если покупатель купил {Хлеб, Молоко}, то он с высокой вероятностью купит и {Масло}".
    *   **Алгоритмы:** Apriori, Eclat.

В этой лекции мы сосредоточимся на **кластеризации**.

### Часть 2: Алгоритм K-Means (Кластеризация на основе центроидов)

K-Means — один из старейших и самых популярных алгоритмов кластеризации. Его идея очень проста: найти `K` центров, вокруг которых группируются данные.

#### Интуиция: Алгоритм Ллойда
Алгоритм работает итеративно:
1.  **Шаг 0: Выбрать `K`** — количество кластеров.
2.  **Шаг 1: Инициализация.** Случайно разместить `K` центроидов.
3.  **Шаг 2: Присвоение.** Каждую точку отнести к ближайшему центроиду.
4.  **Шаг 3: Обновление.** Пересчитать положение каждого центроида как среднее всех точек в его кластере.
5.  **Повторение:** Повторять шаги 2 и 3 до сходимости.

> **Интерактивная визуализация:** Поиграть с алгоритмом K-Means вживую можно здесь: 
> [https://www.naftaliharris.com/blog/visualizing-k-means-clustering/](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

Математически, K-Means пытается минимизировать **сумму квадратов внутрикластерных расстояний Within-Cluster Sum of Squares (WCSS)**, также известную как `inertia`:
$$ \Phi_0 = \sum_{a=1}^{K} \sum_{i: a_i=a} ||x_i - \mu_a||^2 \rightarrow \min $$
где $\mu_a$ — это центр кластера $a$.

#### K-Means в Scikit-Learn

*   **Класс:** `sklearn.cluster.KMeans`
*   **Ключевые гиперпараметры:**
    *   `n_clusters`: То самое `K`, количество кластеров. **Самый важный параметр.**
    *   `init`: Метод инициализации центроидов.
        *   `'random'`: Выбирает `K` случайных точек из датасета в качестве начальных центроидов. Этот метод очень быстрый, но может привести к неудачной сходимости, если начальные центры окажутся близко друг к другу.
        *   `'k-means++'` (по умолчанию): Это "умный" метод инициализации. Он работает так:
            1.  Первый центроид выбирается случайно.
            2.  Каждый следующий центроид выбирается из оставшихся точек с вероятностью, пропорциональной **квадрату расстояния** до ближайшего из уже выбранных центроидов.
            **Простыми словами:** `k-means++` старается выбрать начальные центроиды как можно **дальше друг от друга**. Это значительно повышает шансы на быструю и качественную сходимость алгоритма.
    *   `n_init`: Сколько раз алгоритм будет запущен с разными случайными начальными центроидами. Модель вернет лучший результат (с наименьшим `inertia_`) из всех запусков.
        *   **`'auto'` (по умолчанию с версии 1.4):** Это умная настройка. Если `init='k-means++'`, то будет выполнен только **один** запуск (так как этот метод детерминирован). Если `init='random'`, то будет выполнено **10** запусков. Это значение по умолчанию является отличным выбором в большинстве случаев.
    *   `random_state`: Число для инициализации генератора случайных чисел. Используется для воспроизводимости результатов, так как `k-means++` и `'random'` содержат элемент случайности.

#### Как выбрать `K`? Метод локтя (Elbow Method)

Главный недостаток K-Means — мы должны заранее задать `K`. **Метод локтя** — это популярная эвристика для его поиска:
1.  Мы запускаем K-Means для разного количества кластеров (например, от `K=2` до `10`).
2.  Для каждого `K` мы вычисляем итоговое значение `WCSS` (атрибут `model.inertia_`).
3.  Строим график зависимости `WCSS` от `K`.
4.  Ищем на графике точку, похожую на "локоть" — место, где кривая начинает выполаживаться. Это и есть наш кандидат на оптимальное `K`.


In [ ]:
import os

# Устанавливаем несколько переменных окружения, чтобы покрыть разные библиотеки
# Устанавливаем значение '1', чтобы гарантированно отключить проблемный параллелизм для этой операции
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

# Теперь импортируем все остальное
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [ ]:
# Генерируем данные
X, y = make_blobs(n_samples=500, centers=4, random_state=42, cluster_std=1.0)

ssd = []
k_range = range(1, 11)
for k in k_range:
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    model.fit(X)
    ssd.append(model.inertia_)

# Рисуем график
plt.figure(figsize=(8,6))
plt.plot(k_range, ssd, 'o-')
plt.xlabel("Количество кластеров (K)")
plt.ylabel("Sum of Squared Distances (WCSS)")
plt.title("Метод локтя для K-Means")
plt.xticks(k_range)
plt.grid(True)
plt.show()

### Визуализация результата с оптимальным K

Из графика "метода локтя" мы видим, что оптимальное количество кластеров K=4.
Теперь давайте обучим финальную модель с этим значением и посмотрим на результат.

In [ ]:
# 1. Обучаем финальную модель с K=4
final_model = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = final_model.fit_predict(X)

# Метки кластеров — это массив, где каждой точке присвоен номер кластера (от 0 до 3)
print("Примеры меток кластеров для первых 10 точек:")
print(labels[:10])

# 2. Визуализируем кластеры
plt.figure(figsize=(10, 7))

# Рисуем точки, раскрашивая их в соответствии с метками кластеров
sns.scatterplot(x=X[:,0], y=X[:,1], hue=labels, palette='viridis', s=60)

# Отдельно рисуем центры кластеров, чтобы они были хорошо видны
centers = final_model.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.8, marker='X', label='Центроиды')

plt.title("Результат кластеризации K-Means (K=4)")
plt.xlabel("Признак 1")
plt.ylabel("Признак 2")
plt.legend()
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

# Сначала нужно вычислить матрицу связей (linkage matrix)
linked = linkage(X, method='ward')

# Теперь строим дендрограмму
plt.figure(figsize=(12, 8))
dendrogram(linked,
            orientation='top',
            distance_sort='descending',
            show_leaf_counts=True)
plt.title('Дендрограмма иерархической кластеризации')
plt.ylabel('Расстояние по Уорду')

# Добавим линию среза для выбора K
plt.axhline(y=35, c='k', linestyle='--') 

plt.show()

Проведя горизонтальную линию там, где мы видим самые длинные вертикальные отрезки, мы пересекаем 4 линии, что также указывает на 4 кластера. Это - иерархическая кластеризация.

### Часть 3: Альтернативный подход — Иерархическая кластеризация

Помимо K-Means, существует другой популярный метод, основанный на расстоянии, — **иерархическая кластеризация**. Ее главная особенность в том, что она не просто делит данные на `K` кластеров, а строит целое \"дерево\" вложенных друг в друга кластеров.

#### Интуиция: Агломеративный подход

Самый распространенный вид иерархической кластеризации — **агломеративный** (объединительный). Алгоритм работает "снизу вверх":
1.  **Начало:** Каждая точка данных считается отдельным кластером.
2.  **Шаг 1:** Находятся два **самых близких** друг к другу кластера и объединяются в один.
3.  **Шаг 2:** Снова находятся два самых близких кластера и объединяются.
4.  **Повторение:** Процесс повторяется до тех пор, пока все точки не окажутся в одном большом кластере.

#### Иерархическая кластеризация в Scikit-Learn и SciPy
*   **Класс:** `sklearn.cluster.AgglomerativeClustering`
*   **Ключевые гиперпараметры:**
    *   `n_clusters`: Количество кластеров, которое мы хотим получить в итоге (если `distance_threshold=None`).
    *   `metric` (или `affinity`): Метрика расстояния между точками (`euclidean`, `manhattan`).
    *   `linkage`: **Самый важный параметр.** Правило, по которому измеряется расстояние **между кластерами**.

#### Как работают разные методы `linkage`?

Пусть у нас есть два кластера, $U$ и $V$, и мы хотим измерить расстояние между ними, $D(U, V)$.

![linkage](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/linkage.png)

*   **`'single'` (Расстояние ближнего соседа):**
    *   **Интуиция:** Расстояние между кластерами — это расстояние между **двумя самыми близкими** точками из этих кластеров. Это "оптимистичный" подход, который хорошо находит вытянутые, ленточные кластеры.
    $$ D(U, V) = \min_{u \in U, v \in V} d(u, v) $$

*   **`'complete'` (Расстояние дальнего соседа):**
    *   **Интуиция:** Расстояние между кластерами — это расстояние между **двумя самыми далекими** точками из этих кластеров. Этот "пессимистичный" подход стремится создавать более компактные, шарообразные кластеры.
    $$ D(U, V) = \max_{u \in U, v \in V} d(u, v) $$

*   **`'average'` (Групповое среднее расстояние):**
    *   **Интуиция:** Расстояние — это **среднее арифметическое** всех возможных попарных расстояний между точками из одного кластера и точками из другого. Это компромисс между `single` и `complete`.
    $$ D(U, V) = \frac{1}{|U| \cdot |V|} \sum_{u \in U} \sum_{v \in V} d(u, v) $$

*   **`'ward'` (Метод Уорда):**
    *   **Интуиция:** Этот метод работает иначе. Он объединяет те два кластера, слияние которых приведет к **минимальному увеличению суммарной внутрикластерной дисперсии (WCSS)**. По сути, на каждом шаге он выбирает такое объединение, которое создает наиболее компактный из всех возможных новых кластеров.
    *   Этот метод стремится создавать кластеры примерно одинакового размера и часто является лучшим выбором по умолчанию, если вы ожидаете найти сферические кластеры.

#### Визуализация: Дендрограмма

Весь этот процесс объединения можно визуализировать с помощью **дендрограммы**, которую удобнее всего строить с помощью библиотеки `SciPy`.

**Как читать дендрограмму:**
*   По **вертикали** отложена высота — расстояние (согласно выбранному методу `linkage`), на котором произошло объединение. Длинные вертикальные линии означают, что объединялись очень далекие друг от друга кластеры.
*   **Дендрограмма** — это еще один способ выбрать оптимальное `K`. Мы можем провести горизонтальную линию-срез в том месте, где она пересечет самую длинную вертикальную линию. Количество пересечений и будет рекомендованным числом кластеров.

#### Визуализация: Дендрограмма на практическом примере

Давайте посмотрим, как работает иерархическая кластеризация и дендрограмма на простом и наглядном примере. Создадим небольшой датасет, где есть 12 человек и два признака: "Средние часы работы в день" и "Месячный доход (в тыс.)". Интуитивно мы ожидаем увидеть 3 группы: офисные работники, студенты и фрилансеры.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Создаем наглядный датасет
X = np.array([
    # Группа 1: Офисные работники (много часов, средний доход)
    [8, 150], [9, 160], [8.5, 140], [7.5, 155],
    # Группа 2: Студенты (мало часов, низкий доход)
    [3, 40], [4, 50], [2.5, 35], [3.5, 45],
    # Группа 3: Фрилансеры (гибкие часы, высокий доход)
    [5, 200], [6, 210], [4.5, 190], [5.5, 220]
])

# 1. Визуализируем исходные данные
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X[:,0], y=X[:,1], s=100)
plt.title('Исходные данные: 12 человек')
plt.xlabel('Часы работы в день')
plt.ylabel('Доход (в тыс.)')
plt.grid(True)
plt.show()

# 2. Строим дендрограмму
# linkage вычисляет матрицу связей, 'ward' минимизирует дисперсию при объединении
linked = linkage(X, method='ward')

plt.figure(figsize=(12, 8))
dendrogram(linked,
            orientation='top',
            labels=[f"Точка {i}" for i in range(len(X))], # Подписываем точки
            distance_sort='descending',
            show_leaf_counts=True)

plt.title('Дендрограмма иерархической кластеризации')
plt.ylabel('Расстояние по Уорду')
plt.axhline(y=100, c='k', linestyle='--') # Добавим линию среза
plt.show()

# 3. Применяем AgglomerativeClustering с K=3
agg_cluster = AgglomerativeClustering(n_clusters=3, linkage='ward')
labels = agg_cluster.fit_predict(X)

# 4. Визуализируем результат кластеризации
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X[:,0], y=X[:,1], hue=labels, palette='viridis', s=100)
plt.title('Результат иерархической кластеризации (K=3)')
plt.xlabel('Часы работы в день')
plt.ylabel('Доход (в тыс.)')
plt.grid(True)
plt.show()

**Интерпретация результатов:**

1.  **Исходные данные:** На первом графике мы четко видим 3 визуальных кластера.
2.  **Дендрограмма:**
    *   **Нижние уровни:** Алгоритм сначала объединяет самые близкие точки внутри каждой группы (например, двух студентов с похожими параметрами).
    *   **Средние уровни:** Затем он объединяет эти небольшие подгруппы в три больших кластера. Высота "перемычек" внутри этих кластеров относительно небольшая.
    *   **Верхний уровень:** Чтобы объединить эти 3 больших кластера, алгоритму приходится делать "прыжки" на очень большое расстояние (длинные вертикальные линии).
3.  **Выбор K:** Проведя горизонтальную линию среза (`y=100`) через самую длинную вертикальную линию, мы пересекаем ровно **3** ветви. Это подтверждает нашу интуицию, что оптимальное количество кластеров — **три**.
4.  **Итоговая кластеризация:** Финальный график показывает, что `AgglomerativeClustering` с `n_clusters=3` идеально справился с задачей, в точности выделив те группы, которые мы и ожидали увидеть.

### Часть 4: Где K-Means терпит неудачу?

K-Means очень эффективен, но у него есть фундаментальное ограничение: он предполагает, что кластеры имеют **сферическую (выпуклую) форму** и примерно одинаковый размер. Он терпит неудачу на кластерах сложной формы.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_moons, make_circles
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Генерируем данные
# Датасет "полумесяцы"
X_moons, y_moons = make_moons(n_samples=500, noise=0.1, random_state=42)
# Датасет "кольца"
X_circles, y_circles = make_circles(n_samples=500, factor=0.5, noise=0.05, random_state=42)

# Масштабируем данные, так как K-Means основан на расстоянии
X_moons_scaled = StandardScaler().fit_transform(X_moons)
X_circles_scaled = StandardScaler().fit_transform(X_circles)

# 2. Обучаем K-Means для каждого набора данных
# Мы знаем, что в обоих случаях должно быть 2 кластера, поэтому n_clusters=2
kmeans_moons = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans_circles = KMeans(n_clusters=2, random_state=42, n_init=10)

# Получаем метки кластеров
labels_moons = kmeans_moons.fit_predict(X_moons_scaled)
labels_circles = kmeans_circles.fit_predict(X_circles_scaled)

# 3. Визуализируем результаты
# Создаем область для двух графиков рядом
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# График для "полумесяцев"
sns.scatterplot(ax=axes[0], x=X_moons[:,0], y=X_moons[:,1], hue=labels_moons, palette='viridis', s=50)
axes[0].set_title('Неудачный результат K-Means на \"полумесяцах\"')
axes[0].set_xlabel('Признак 1')
axes[0].set_ylabel('Признак 2')


# График для "колец"
sns.scatterplot(ax=axes[1], x=X_circles[:,0], y=X_circles[:,1], hue=labels_circles, palette='viridis', s=50)
axes[1].set_title('Неудачный результат K-Means на \"кольцах\"')
axes[1].set_xlabel('Признак 1')
axes[1].set_ylabel('Признак 2')


plt.suptitle('Ограничения K-Means: кластеры нелинейной формы', fontsize=16)
plt.show()

### Часть 5: Алгоритм DBSCAN (Кластеризация на основе плотности)

DBSCAN предлагает совершенно другой подход. Вместо поиска центров, он ищет **плотные скопления точек**.

> **Интерактивная визуализация:** Поиграть с алгоритмом DBSCAN вживую можно здесь: 
> [https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/)

#### Ключевые понятия и гиперпараметры
DBSCAN определяется двумя гиперпараметрами:
1.  **`eps` (эпсилон):** Радиус окрестности. Это расстояние, в пределах которого мы ищем соседей.
2.  **`min_samples`:** Минимальное число соседей (включая саму точку) в `eps`-окрестности, чтобы точка считалась "ядром плотности".

#### DBSCAN в Scikit-Learn
*   **Класс:** `sklearn.cluster.DBSCAN`
*   **Ключевые гиперпараметры:** `eps` и `min_samples`.
  
На основе этих параметров все точки делятся на три типа:

![dbscan](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/dbscan.png)

*   **Core Point (Ядровая):** Точка, у которой в `eps`-окрестности есть не менее `min_samples` соседей. Это "сердце" кластера.
*   **Border Point (Граничная):** Точка, у которой соседей меньше, чем `min_samples`, но она сама является соседом какой-либо ядровой точки. Это "край" кластера.
*   **Noise (Шум/Выброс):** Точка, которая не является ни ядровой, ни граничной. Она лежит в разреженной области.

#### Алгоритм на пальцах
1.  Выбирается случайная, еще не посещенная точка.
2.  Если она **ядровая**, то она начинает **новый кластер**.
3.  Этот кластер начинает "расти": все ее соседи добавляются в этот же кластер. Если кто-то из соседей тоже оказывается ядровой точкой, то и его соседи тоже присоединяются. Процесс продолжается, пока кластер не перестанет расти.
4.  Если точка **не ядровая**, она временно помечается как "шум". Позже она может стать граничной, если окажется в окрестности какого-то другого кластера.
5.  Процесс повторяется для всех точек.

In [ ]:
# Пример, где K-Means плох, а DBSCAN хорош
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=500, noise=0.1, random_state=42)
X_moons_scaled = StandardScaler().fit_transform(X_moons)

# Модели
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
dbscan = DBSCAN(eps=0.3)

# Предсказания
labels_kmeans = kmeans.fit_predict(X_moons_scaled)
labels_dbscan = dbscan.fit_predict(X_moons_scaled)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.scatterplot(ax=axes[0], x=X_moons[:,0], y=X_moons[:,1], hue=labels_kmeans, palette='viridis')
axes[0].set_title('Неудачный результат K-Means')

sns.scatterplot(ax=axes[1], x=X_moons[:,0], y=X_moons[:,1], hue=labels_dbscan, palette='viridis')
axes[1].set_title('Успешный результат DBSCAN')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_circles  # Заменяем make_moons на make_circles
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

# 1. Генерируем данные в виде двух колец
# factor=0.5 означает, что внутреннее кольцо будет в два раза меньше внешнего
X_circles, y_circles = make_circles(n_samples=500, noise=0.05, factor=0.5, random_state=42)

# Масштабируем данные, что важно для DBSCAN, так как eps - это мера расстояния
X_circles_scaled = StandardScaler().fit_transform(X_circles)

# 2. Инициализируем модели
# Для K-Means мы знаем, что кластеров должно быть 2
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
# Для DBSCAN подбираем eps. Для компактных колец он должен быть меньше, чем для лун.
dbscan = DBSCAN(eps=0.3)

# 3. Обучаем модели и получаем метки кластеров
labels_kmeans = kmeans.fit_predict(X_circles_scaled)
labels_dbscan = dbscan.fit_predict(X_circles_scaled)

# 4. Визуализируем результаты
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# График для K-Means
sns.scatterplot(ax=axes[0], x=X_circles[:,0], y=X_circles[:,1], hue=labels_kmeans, palette='viridis')
axes[0].set_title('Неудачный результат K-Means на "кольцах"')
axes[0].set_xlabel("Признак 1")
axes[0].set_ylabel("Признак 2")

# График для DBSCAN
sns.scatterplot(ax=axes[1], x=X_circles[:,0], y=X_circles[:,1], hue=labels_dbscan, palette='viridis')
axes[1].set_title('Успешный результат DBSCAN на "кольцах"')
axes[1].set_xlabel("Признак 1")
axes[1].set_ylabel("Признак 2")

plt.suptitle('Сравнение K-Means и DBSCAN на данных в виде колец', fontsize=16)
plt.show()

### Часть 6: Интерпретация кластеров — нетривиальная задача

Важно понимать, что алгоритмы кластеризации возвращают лишь **номера кластеров** (0, 1, 2, ...). Они не говорят нам, **что означают** эти группы. Присвоение бизнес-смысла полученным кластерам — это отдельная и очень важная задача, которая называется **пост-обработкой** или **интерпретацией**.

**Основной подход:**
1.  **Добавить метки:** Добавить полученные метки кластеров в исходный, **немасштабированный** DataFrame.
2.  **Профилирование кластеров:** Использовать `groupby('cluster').mean()` или `.describe()` для анализа средних значений и распределений признаков внутри каждого кластера.
3.  **Создание "персон":** На основе анализа дать каждому кластеру осмысленное название. Например, если мы кластеризуем клиентов интернет-магазина, у нас могут получиться:
    *   **Кластер 0 ("Лояльные VIP-клиенты"):** Высокий средний чек, высокая частота покупок, большой `total_spent`.
    *   **Кластер 1 ("Охотники за скидками"):** Низкий средний чек, покупки только в период распродаж.
    *   **Кластер 2 ("Новички"):** Мало покупок, низкий `total_spent`.

Эта интерпретация позволяет превратить математический результат в **полезный для бизнеса инструмент**.

### Часть 7: Итоговые выводы по лекции

1.  Мы перешли от **Обучения с учителем** к **Обучению без учителя**, где главная задача — найти **скрытую структуру** в данных, а не предсказать известную метку.

2.  Мы рассмотрели **K-Means** — быстрый и простой алгоритм, который ищет **сферические кластеры** на основе центроидов. Его главный недостаток — необходимость заранее задавать количество кластеров `K`, которое можно оценить с помощью **метода локтя**.

3.  Мы кратко коснулись **Иерархической кластеризации** и **дендрограмм** — мощного визуального инструмента, который также помогает в определении числа кластеров.

4.  Мы изучили **DBSCAN** — алгоритм, основанный на **плотности**. Он не требует заранее знать число кластеров, умеет находить кластеры **произвольной формы** и автоматически определяет **выбросы**.

5.  Главный вывод: **не существует универсально "лучшего" алгоритма кластеризации**. Выбор метода зависит от структуры ваших данных и целей исследования.